In [1]:
!jupyter nbextension enable --py widgetsnbextension
import random as rd
import os
import torch
from PIL import Image
from tqdm import tqdm
import numpy as np
from datasets import load_dataset,load_metric,DatasetDict,concatenate_datasets,config
from datasets import Image as DImage
from transformers import ViTFeatureExtractor, ViTModel, ViTForImageClassification, TrainingArguments, Trainer, integrations

device = "cuda" if torch.cuda.is_available() else "cpu"

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [ ]:
from datasets import load_dataset,load_metric,DatasetDict,Dataset,Image
ds = load_dataset("imagefolder",data_dir='./Data')
labels = ds['train'].features['label'].names

In [3]:
display(labels)
def same_label(example,lbl):
    ls = []
    # display(example)
    for val in example['label']:
        ls.append(val==lbl)
    return ls

['bacterial_spot',
 'black_rot',
 'black_spot',
 'blight',
 'brown_spot_fungi',
 'brown_streak',
 'canker',
 'fungi_spot',
 'gray_spot',
 'green_mottle',
 'greening',
 'healthy',
 'hispa',
 'isariopis_spot',
 'leaf_blast',
 'measeles',
 'mold',
 'mosaic',
 'powdery_mildew',
 'rust',
 'scab',
 'scorch',
 'spider_mites',
 'target_spot',
 'yellow_curl']

In [9]:
#[labels[lb] for lb in range(20,len(labels))]

['scab', 'scorch', 'spider_mites', 'target_spot', 'yellow_curl']

In [ ]:
ds_list = DatasetDict()
# for label in (range(20,len(labels))):
for label in (range(0,len(labels))):
    ds_list[labels[label]]=ds.filter(same_label, batched=True,batch_size=10000,fn_kwargs={'lbl':label},num_proc =8)['train']
ds_list

In [ ]:
# import torchvision.transforms as ttr
# # from math import floor

# map_fun = ttr.Compose(
#     [ttr.RandomHorizontalFlip(),ttr.RandomRotation(180,fill=255),ttr.GaussianBlur(9,sigma=(0.01,2.0))]
# )

# def map_red_fun(exp_batch,times=1,mp_fn=map_fun):# batch: Dict[str, List]) -> Dict[str, List]
#     itms = int(times)
#     ltms = int((times-int(times))*len(exp_batch['label']))
#     print("times:",itms,ltms)#,times,times-floor(times),len(exp_batch['label']))
    
    
#     if(times<1):
#         # return {
#         # 'image': exp_batch['image'][:ltms]
#         # 'label': exp_batch['label'][:ltms]
#         # }
#         return exp_batch[:ltms]
#     elif(times==1):
#         return exp_batch
#     else:
#         images = list(exp_batch['image'])
#         lbls = exp_batch['label']
#         if itms>1:    
#             for i in range(itms-1):
#                 images += [mp_fn(img).convert('RGB') for img in exp_batch['image']]
#             lbls += exp_batch['label']*(itms-1)
#         if(ltms>0):
#             images += [mp_fn(img).convert('RGB') for img in exp_batch['image'][:ltms]]
#             lbls += exp_batch['label'][:ltms]
    
#         return {
#             'image':images,
#             'label':lbls
#         }
        

In [ ]:
files = os.listdir("./Data/")
for lbl in files:
    if not os.path.isdir("./NData/"+lbl+"/"):
        os.mkdir("./NData/"+lbl+"/")

In [11]:
import torchvision.transforms as ttr
# from math import floor

map_fun = ttr.Compose(
    [ttr.RandomHorizontalFlip(),ttr.RandomRotation(180,fill=255),ttr.GaussianBlur(9,sigma=(0.01,2.0))]
)

def save_fun(zhelabel,img,ldx):
    img.save("./NData/"+zhelabel+"/Augmented_"+str(ldx)+".png")

def map_save_fun(exp_batch,idxs,zhelabel,times=1,sv_fn=save_fun,mp_fn=map_fun):# batch: Dict[str, List]) -> Dict[str, List]
    itms = int(times)
    ltms = int((times-int(times))*len(exp_batch['label']))
    print("times:",itms,ltms)#,times,times-floor(times),len(exp_batch['label']))
    ldx= idxs[0]*100000
    
    if(times<=1):
        # return {
        # 'image': exp_batch['image'][:ltms]
        # 'label': exp_batch['label'][:ltms]
        # }
        for idx,img in enumerate(exp_batch['image']):
            if(idx<ltms):
                sv_fn(zhelabel,img.convert('RGB'),ldx)
                ldx+=1
            
        # return exp_batch[:ltms]
    else:
        images = list(exp_batch['image'])
        for img in exp_batch['image']:
            sv_fn(zhelabel,img.convert('RGB'),ldx)
            ldx+=1
        
        if itms>1:    
            for i in range(itms-1):
                for img in exp_batch['image']:
                    sv_fn(zhelabel,mp_fn(img).convert('RGB'),ldx)
                    ldx+=1
                # images += [mp_fn(img).convert('RGB') for img in exp_batch['image']]
        if(ltms>0):
            for idx,img in enumerate(exp_batch['image']):
                if(idx<ltms):
                    sv_fn(zhelabel,mp_fn(img).convert('RGB'),ldx)
                    ldx+=1
            # images += [mp_fn(img).convert('RGB') for img in exp_batch['image'][:ltms]]

    
        # return {
        #     'image':images,
        #     'label':lbls
        # }
    return exp_batch



In [14]:
labels[20]

'scab'

In [ ]:
# for dss in labels[20:]:
for dss in labels:
    # display(str(ds_list[dss].shape[0])+' : '+str(40000/ds_list[dss].shape[0])+' = '+str(int(40000/ds_list[dss].shape[0]
    #                                                                                         ) + int((40000/ds_list[dss].shape[0]-int(40000/ds_list[dss].shape[0]))*len(ds_list[dss].shape[0]))))
    # aug_ds_list[dss]=ds_list[dss].map(map_red_fun,batched=True,batch_size=40,num_proc=8,fn_kwargs={'times':40000/ds_list[dss].shape[0],'mp_fn':map_fun})
    ds_list[dss].map(map_save_fun, batched=True, with_indices=True, batch_size=40, num_proc=8, fn_kwargs={
                     'times': 40000/ds_list[dss].shape[0], 'sv_fn': save_fun, 'zhelabel': dss,'mp_fn':map_fun})
    # display(aug_ds_list[dss])
# aug_ds_list


In [ ]:
dss = labels[20]
ds_list[dss].map(map_save_fun, batched=True, with_indices=True, batch_size=10, num_proc=4, fn_kwargs={
                     'times': 40000/ds_list[dss].shape[0], 'sv_fn': save_fun, 'zhelabel': dss,'mp_fn':map_fun})